<a href="https://colab.research.google.com/github/Pallavi-Kolambkar/NEU_project/blob/master/BEL_to_SCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp27-cp27mu-linux_x86_64.whl 
!pip install pyro-ppl

ERROR: torch-0.4.0-cp27-cp27mu-linux_x86_64.whl is not a supported wheel on this platform.
     |████████████████████████████████| 522kB 2.8MB/s 


In [0]:
!pip3 install flake8

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 


In [0]:
! flake8 -help

Usage: flake8 [options] file file ...

Options:
  --version             show program's version number and exit
  -h, --help            show this help message and exit
  -v, --verbose         Print more information about what is happening in
                        flake8. This option is repeatable and will increase
                        verbosity each time it is repeated.
  -q, --quiet           Report only file names, or nothing. This option is
                        repeatable.
  --count               Print total number of errors and warnings to standard
                        error and set the exit code to 1 if total is not
                        empty.
  --diff                Report changes only within line number ranges in the
                        unified diff provided on standard in by the user.
  --exclude=patterns    Comma-separated list of files or directories to
                        exclude. (Default:
                        .svn,CVS,.bzr,.hg,.git,__pycache__,.tox,

The project aims to create a causal model for the host 

In [0]:
#checking if the comment works
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx
# from pybel.examples import sialic_acid_graph as sag
# import pybel as pb
import json
import time
import csv
# from pybel.io.jupyter import to_jupyter
import torch
import pyro
import pandas as pd
import numpy as np
pyro.set_rng_seed(101)

In [0]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [0]:
# from google.colab import files 
# uploaded = files.upload() 
# f = open('COVID19.jgf')
# bel_statements = json.load(f)

Saving COVID-19-new.json to COVID-19-new.json


FileNotFoundError: ignored

In [0]:
# type has the structure of (type_parent, type_children, type_relation):[[edges]]
label_dict = {'Abundance':['a', 'r', 'm', 'g', 'p','pop', 'composite', 'complex','frag','fus','loc','pmod','var'],
             'Process': ['bp', 'path','act'],
             'Transformation':['sec','surf','deg','rxn','tloc','fromLoc','products','reactants','toLoc']}
def get_information(jgf_file):
    # causal_relations = ['increases', 'decreases', 'directlyIncreases', 'directlyDecreases']
    types = []
    # nodes = set()
    parents = []
    parent_types = []
    children_types = []
    parent_labels = []
    children_labels = []
    children = []
    str_list = []
    df = pd.DataFrame()
    
    for edges in jgf_file['graph']['edges']:
        parent = edges['source']
        child = edges['target']
        relation = edges['relation']
        text = edges['label']
        if relation.find('crease') > 0:
            # nodes.add(parent)
            # nodes.add(child)
            parents.append(parent)
            children.append(child)
            str_list.append(parent + '*' + relation + '*' + child)   
            parent_type = parent[:parent.find('(')]
            children_type = child[:child.find('(')]
            types.append(relation)
            children_types.append(children_type)
            parent_types.append(parent_type)
            for label in label_dict:
                if parent_type in label_dict[label]:
                    parent_labels.append(label)
                elif parent_type == '': 
                    parent_labels.append('Others')
            for label in label_dict:
                if children_type in label_dict[label]:     
                    children_labels.append(label)
                elif children_type == '': 
                    children_labels.append('Others')
                    break
            # types.setdefault((parent_type,chidren_type,relation), []).append(parent + '*' + relation + '*' + child)
    df['parents'] = parents
    df['children'] = children
    df['types'] = types
    df['statements'] = str_list
    df['parent_types'] = parent_types
    df['parent_labels'] = parent_labels
    df['children_types'] = children_types
    df['children_labels'] = children_labels
    
    return df

In [0]:
df = get_information(bel_statements)
df = df[~df['children_labels'].isin(['Others'])] # filter the two with nested structures

In [0]:
df.head()

In [0]:
## identifying clusters
new_df = df.groupby(['children', 'types', 'children_types','children_labels']).apply(lambda x: [list(x['parents']), list(x['parent_types']),list(x['parent_labels'])]).apply(pd.Series)

In [0]:
new_df = new_df.reset_index(drop = False)
new_df.columns = ['children', 'types', 'children_types','children_labels', 'parents', 'parents_types','parent_labels']

In [0]:
new_df

In [0]:
new_df.iloc[16]

children           p(HGNC:ANGPT2)                                                                    
types              decreases                                                                         
children_types     p                                                                                 
children_labels    Abundance                                                                         
parents            [p(HGNC:VWF), act(complex(SCOMP:"AMP Activated Protein Kinase Complex"), ma(kin))]
parents_types      [p, act]                                                                          
parent_labels      [Abundance, Process]                                                              
Name: 16, dtype: object

# For each row we need to build a SCM 

## Jeremy's guide for dealing with node types

Let's talk about Abundance -> Process first.   If you have multiple Abundance parents, then the function could simply be a weighted sum of abundances followed by an activation function to determine whether the process is on or off.  In the simple case where you have two abundance parents, the Parent-Child function resembles an OR gate if the weights are large, because either abundance (or both) is sufficient to activate the process.  If the weights are small, then the the Parent-Child function resembles an AND gate, because you need both parents in high abundance.


7:20
Let's talk about  Process -> Transformation next.    In this case, if the process is active, then the transformation is a continuous variable.  If the process in inactive, then the transformation cannot occur, so it is zero.


7:21
Let's talk about (Abundance*, Process) -> Transformation.  In this case, the transformation is a potentially nonlinear function of abundance if the Process is active, but zero otherwise.


7:23
if Transformation -> Abundance, then we have a continuous in, and continuous out, but it may be nonlinear.


7:24
Process -> Abundance implies  continuous if Process is active and 0 otherwise.


7:25
All of these rules are flipped if the causal relation is decreases instead of increases


7:26
If Abundance -> Transformation we also have (potentially nonlinear) continuous to continuous.


In [0]:
import pyro.distributions as dist
def get_distribution(node_type):
    if node_type == 'Abundance':
        return dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))
    if node_type == 'Process':
        return dist.Categorical(torch.tensor([0.5,0.5]))
    else:
        return dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))

In [0]:
def SCM(parent_label,child_label,relation,threshold,w):
  parents = []
  process = []
  abundance = []
  transformation = []
  for i in range(len(parent_label)):
    parent = pyro.sample("par_%s" % i, get_distribution(parent_label[i]))
    parents.append(parent)
    if parent_label[i] =='Process':
      process.append(parent)
    if parent_label[i] == 'Abundance':
      abundance.append(parent)
    if parent_label[i] == 'Transformation':
        transformation.append(parent)


  if child_label != 'Transformation':
    
    if relation == 'increases':
      ## process is more like an OR switch
      if sum(process) > 0 :
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
        # + sum([x*y*y for x,y in zip(w ,transformation)])
      else: 
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      if child_label == 'Process':
        check = lambda x : 1.0 if x > threshold else 0.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
      
    if relation == 'decreases':
      if sum(process) > 0:
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
      else:
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
       # + sum([x*y*y for x,y in zip(w ,transformation)])
      if child_label == 'Process':
        check = lambda x : 0.0 if x > threshold else 1.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
    

  else:
    if relation == 'increases':
      ## process is more like an OR switch
      if sum(process) > 0 :
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) + \
        sum([x*y*y for x,y in zip(w ,transformation)])
      else: 
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      if child_label == 'Process':
        check = lambda x : 1.0 if x > threshold else 0.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
      
    if relation == 'decreases':
      if sum(process) > 0:
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      else:
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) + \
        sum([x*y*y for x,y in zip(w ,transformation)])
      if child_label == 'Process':
        check = lambda x : 0.0 if x > threshold else 1.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))

#     if relation == 'increases':
#       if sum(process) > 0:
#         child_N =  sum([x*y*y for x,y in zip(w ,parents)]) + pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       else: 
#         child_N = pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       child = pyro.sample('child',pyro.distributions.Delta(child_N))
#       # return{'parents': parents,'child': child}

#     if relation == 'decreases':
#       if sum(process) > 0:
#         child_N = pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       else:
#         child_N =  sum([x*y*y for x,y in zip(w ,parents)]) + pyro.sample("child_n", pyro.distributions.Normal(0, 1))
#       child = pyro.sample('child',pyro.distributions.Delta(child_N))
  #return child.numpy()
  return{'parents': parents,'child': child}

In [0]:
# get the model for each row
SCM_list = []
for i in range(new_df.shape[0]):
    if i in [2,3,4]:
      continue
    test = new_df.iloc[i]
    SCM_list.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[3]*len(test['parent_labels'])))
SCM_list
len(SCM_list)

42

# Adding a KL Divergence test to check the correctness of distributions we get from SCM against the true distribution of our child nodes. 

In [0]:
# get the model for each row
#SCM_list23 = []
#SCM_list22 = []
#SCM_list33 = []
#true_children = []
#for i in range(new_df.shape[0]):
#    if i in [2,3,4]:
#        continue
#    test = new_df.iloc[i]
#    SCM_list22.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[2]*len(test['parent_labels'])))
#    SCM_list23.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[3]*len(test['parent_labels'])))
#    SCM_list33.append(SCM(test['parent_labels'],test['children_labels'],test['types'],3,[3]*len(test['parent_labels'])))
#    true_children.append(pyro.sample("child", get_distribution(test['children_labels'])))

In [0]:
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
def KL(P,Q):
    """ Epsilon is used here to avoid conditional code for
    checking that neither P nor Q is equal to 0. """
    epsilon = 0.00001

    # You may want to instead make copies to avoid changing the np arrays.
    P = P+epsilon
    Q = Q+epsilon

    divergence = np.sum(P*np.log(P/Q))
    return divergence

In [0]:
print(KL(np.asarray(SCM_list23[5]), np.asarray(true_children[5])))

-0.0001151293546492023


#### Linear regression to get weights from SCM


In [0]:
#import pandas as pd
#samples_df = pd.DataFrame(SCM_list22)
#samples_df.head()
#samples_df.tail()

,parents,child
37,[tensor(1.0003)],tensor(3.7504)
38,[tensor(0.4245)],tensor(2.8604)
39,[tensor(0.3862)],tensor(1.3235)
40,[tensor(1.8983)],tensor(-0.4505)
41,[tensor(1.8402)],tensor(-1.0768)


In [0]:
#parent_matrix = []
#child_matrix = []
#for i in range(len(samples_df)):
#    parent_row = [0] * len((max(new_df['parent_labels'], key=len)))
#    for j in range(len(samples_df['parents'][i])):
#        parent_row[j] = samples_df['parents'][i][j].numpy().item()
#    parent_matrix.append(parent_row)
#    child_matrix.append(samples_df['child'][i].numpy().item())

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
regressor = LinearRegression()  
#regressor.fit(parent_matrix, child_matrix) #training the algorithm

In [0]:
#To retrieve the intercept:
#print(regressor.intercept_)
#For retrieving the slope:
#print(regressor.coef_)

In [0]:
samples_2= []
test = new_df.iloc[16]
for i in range(100):
  samples_2.append((SCM(test['parent_labels'],test['children_labels'],test['types'],2,[2]*len(test['parent_labels']))))

In [0]:
samples_2_df.head()

,parents,child,parent1,parent2
0,"[tensor(0.3249), tensor(0)]",tensor(2.2323),tensor(0.3249),tensor(0)
1,"[tensor(6.4018), tensor(1)]",tensor(0.4974),tensor(6.4018),tensor(1)
2,"[tensor(2.5888), tensor(0)]",tensor(5.5890),tensor(2.5888),tensor(0)
3,"[tensor(0.7808), tensor(0)]",tensor(2.8100),tensor(0.7808),tensor(0)
4,"[tensor(0.8992), tensor(1)]",tensor(1.1178),tensor(0.8992),tensor(1)


In [0]:
samples_2_df=pd.DataFrame(samples_2)
samples_2_df[['parent1','parent2']] = pd.DataFrame(samples_2_df['parents'].values.tolist(), index= samples_2_df.index)

In [0]:
sample2_fit=regressor.fit(samples_2_df[['parent1','parent2']].values,samples_2_df['child'])
sample2_fit

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#To retrieve the intercept:
print(sample2_fit.intercept_)
#For retrieving the slope:
print(sample2_fit.coef_)

2.754476079811798
[ 1.56884789 -3.67540574]


#Counterfactual function

In [0]:
# counterfactual function to get the distribution of target variable by sampling
def counterfactual(model,observed_variable, observed_val,exogenous_variable,cf_variable,cf_val,target_variable):
    target = []
    # get the conditional model
    conditions = {}
    for i in range(len(observed_variable)):
        conditions[observed_variable[i]] = torch.tensor(observed_val[i])
    conditioned_model = pyro.condition(model, data=conditions)
    # get the posterior and marginal distribution
    posterior = pyro.infer.Importance(conditioned_model, num_samples=1000).run()
    marginal = EmpiricalMarginal(posterior,exogenous_variable)
    # get the do model
    dos = {}
    for i in range(len(cf_variable)):
        dos[cf_variable[i]] = torch.tensor(cf_val[i])
    do_model = pyro.do(model, data=dos)
    # sample target variable under updated posterior and do model
    for i in range(1000):
        samples = marginal()
        exog = {}
        for i in range(len(exogenous_variable)):
            exog[exogenous_variable[i]] = torch.tensor(samples[i])
        cf_model = pyro.condition(do_model, data= exog)
        trace_handler = pyro.poutine.trace(cf_model)
        trace = trace_handler.get_trace()

        target.append(trace.nodes[target_variable]['value'].numpy().item())

    return target

## To do:
1. code the SCM for the new data (with 16 nodes) Zishen:
    - there are three facts we need to pay attention when building the model, which need to be confirmed by going through the meeting recording 
2. answer counterfactual questions mentioned in the meeting recording (some one need to go through it) Pallavi and Somya
3. get the bio-context of the new graph Somya Pallavi


# Looking into the new data

In [0]:
from google.colab import files 
uploaded = files.upload()
with open('COVID-19-new.json') as f:
    covid19kg_dict=json.load(f)

Saving COVID-19-new.json to COVID-19-new.json


In [0]:
#adding the new file data

parents=[]
children=[]
edgetype=[]
parent_type=[]
children_type=[]
parent_labels = []
children_labels = []
nodes = set()

label_dict = {'Abundance':['a', 'r', 'm', 'g', 'p','pop', 'composite', 'complex','frag','fus','loc','pmod','var'],
             'Process': ['bp', 'path','act'],
             'Transformation':['sec','surf','deg','rxn','tloc','fromLoc','products','reactants','toLoc']}
for eachedge in covid19kg_dict[0]['nanopub']['assertions']:
    parents.append(eachedge['subject'])
    children.append(eachedge['object'])
    edgetype.append(eachedge['relation'])
    nodes.add(eachedge['subject'])
    nodes.add(eachedge['object'])
    p=eachedge['subject']
    c=eachedge['object']
    ptype = p[:p.find('(')]
    ctype = c[:c.find('(')]
    parent_type.append(ptype)
    children_type.append(ctype)
    for label in label_dict:
      if ptype in label_dict[label]:
        parent_labels.append(label)
      elif ptype == '': 
        parent_labels.append('Others')
    for label in label_dict:
      if ctype in label_dict[label]:     
        children_labels.append(label)
      elif ctype == '': 
        children_labels.append('Others')
        breal


In [0]:
newcovid_df=pd.DataFrame(list(zip(parents,parent_type,edgetype,children,children_type, parent_labels, children_labels)),columns=['Parent','Parent_type','Relation','Child','Child_type', 'Parent_label', 'Child_label'])

In [0]:
ailias_dict = {'bp(GO:"tumor necrosis factor-mediated signaling pathway")': 'TNF',
               'bp(GO:"epidermal growth factor receptor signaling pathway")': 'EGFR',
               'act(p(HGNC:EGF))':'EGF',
               'act(p(HGNC:ADAM17))': 'ADAM17',
               'bp(GO:"positive regulation of interleukin-6 production")': 'IL_6_AMP',
               'act(complex(GO:"NF-kappaB complex"))': 'NF_xB',
               'bp(GO:"pattern recognition receptor signaling pathway")': 'PRR',
               'a(TAX:"Severe acute respiratory syndrome coronavirus 2")': 'SARS_2',
               'act(complex(p(HGNC:IL6),p(HGNC:STAT3)))': 'IL6_STAT3',
               'act(p(HGNC:IL6R))': 'IL_6',
               'act(p(HGNC:AGTR1))': 'ATIR',
               'a(CHEBI:"angiotensin II")': 'AngII',
               'act(p(HGNC:ACE2))': 'ACE2',
               'deg(p(HGNC:ACE2))': 'deg',
               'path(MESH:"Severe Acute Respiratory Syndrome")': 'ARDS',
               'act(complex(GO:"NF-kappaB p50/p65 complex"))': 'NF_xB' }
newcovid_df['Parent_alias'] = newcovid_df['Parent'].apply(lambda x : ailias_dict[x])
newcovid_df['Child_alias'] = newcovid_df['Child'].apply(lambda x : ailias_dict[x])
nodes = [ailias_dict[i] for i in nodes]

In [0]:
newcovid_df.sort_values('Parent').head()

,Parent,Parent_type,Relation,Child,Child_type,Parent_label,Child_label,Parent_alias,Child_alias
13,"a(CHEBI:""angiotensin II"")",a,increases,act(p(HGNC:AGTR1)),act,Abundance,Process,AngII,ATIR
8,"a(TAX:""Severe acute respiratory syndrome coronavirus 2"")",a,increases,"bp(GO:""pattern recognition receptor signaling pathway"")",bp,Abundance,Process,SARS_2,PRR
16,"a(TAX:""Severe acute respiratory syndrome coronavirus 2"")",a,increases,deg(p(HGNC:ACE2)),deg,Abundance,Transformation,SARS_2,deg
6,"act(complex(GO:""NF-kappaB complex""))",act,increases,"bp(GO:""positive regulation of interleukin-6 production"")",bp,Process,Process,NF_xB,IL_6_AMP
9,"act(complex(p(HGNC:IL6),p(HGNC:STAT3)))",act,increases,"bp(GO:""positive regulation of interleukin-6 production"")",bp,Process,Process,IL6_STAT3,IL_6_AMP


In [0]:
def SCM_model():
#   ## structural causal model
# def struct_model2(exogenous_dists):
#     X = sample("X", exogenous_dists["N_x"])
#     #Ny_dist = dist.Normal(0.0, 1.0)
#     Y_dist = dist.TransformedDistribution(exogenous_dists["N_y"], AffineTransform(4*X, torch.tensor(1.0)))
#     Y = pyro.sample('Y', Y_dist)
#     return {'X': X, 'Y': Y}

# Y = 4 * X + pyro.sample('Ny', dist.Normal(0.0, 1.0))
# Y = pyro.sample('Y', dist.Normal(Y, 0.01))


  threshold = 0.5
  # the threshold should be determined carefully (e.g. 0.5)
  check_increase = lambda x : 1.0 if x > threshold else 0.0
  check_decrease = lambda x : 0.0 if x > threshold else 1.0
  lognormal_dist = dist.LogNormal(torch.tensor(0.0), torch.tensor(0.5))
  lognormal_dist_noise = dist.Gamma(torch.tensor(0.5),torch.tensor(2.))
  normal_dist = dist.Normal(torch.tensor(0.0),torch.tensor(0.1))
  samples = {}

  SARS_N = pyro.sample('SARS_N',lognormal_dist) 
  # SARS_2 = pyro.sample('SARS_2',pyro.distributions.Delta(SARS_N))
  SARS_2 = pyro.sample('SARS_2',dist.Normal(SARS_N, 0.01))
  samples['SARS_2'] = SARS_2
  
  PRR_N = pyro.sample("PRR_N", normal_dist)
  PRR = torch.tensor(check_increase(PRR_N + SARS_2))
  PRR = pyro.sample('PRR',pyro.distributions.Delta(PRR))
  # PRR = pyro.sample('PRR',dist.TransformedDistribution(lognormal_dist,AffineTransform(PRR, torch.tensor(1.0))))
  # PRR = pyro.sample('PRR',pyro.distributions.Normal(PRR, 0.01))
  samples['PRR'] = PRR

  deg_N = pyro.sample("deg_N", normal_dist)
  deg = SARS_2 + SARS_2*SARS_2 + deg_N
  # deg = pyro.sample('deg',pyro.distributions.Delta(deg))
  # deg = pyro.sample('deg',dist.TransformedDistribution(lognormal_dist,AffineTransform(deg, torch.tensor(1.0))))
  deg = pyro.sample('deg',pyro.distributions.Normal(deg, 0.01))
  samples['deg'] = deg

  ACE2_N = pyro.sample("ACE2_N", normal_dist)
  ACE2 = torch.tensor(check_decrease(deg + ACE2_N))
  ACE2 = pyro.sample('ACE2',pyro.distributions.Delta(ACE2)) 
  samples['ACE2'] = ACE2 

  AngII_N = pyro.sample('AngII_N',lognormal_dist_noise)
  if ACE2 == 0:
    AngII = pyro.sample('AngII',lognormal_dist)
  else: 
    # AngII = pyro.sample('AngII',pyro.distributions.Delta(AngII_N))
    # AngII = pyro.sample('AngII',dist.TransformedDistribution(lognormal_dist,AffineTransform(AngII_N, torch.tensor(1.0))))
    AngII = pyro.sample('AngII',pyro.distributions.Normal(AngII_N, 0.01))
  samples['AngII'] = AngII
  
  ATIR_N = pyro.sample("ATIR_N", normal_dist)
  ATIR = torch.tensor(check_increase(AngII + ATIR_N))
  ATIR = pyro.sample('ATIR',pyro.distributions.Delta(ATIR))
  samples['ATIR'] = ATIR

  ADAM17_N = pyro.sample("ADAM17_N", normal_dist)
  ADAM17 = torch.tensor(check_increase(ATIR + ADAM17_N))
  ADAM17 = pyro.sample('ADAM17',pyro.distributions.Delta(ADAM17))
  samples['ADAM17'] = ADAM17

  TNF_N = pyro.sample("TNF_N", normal_dist)
  TNF = torch.tensor(check_increase(ADAM17 + TNF_N))
  TNF = pyro.sample('TNF',pyro.distributions.Delta(TNF))
  samples['TNF'] = TNF

  EGF_N = pyro.sample("EGF_N", normal_dist)
  EGF = torch.tensor(check_increase(ADAM17 + EGF_N))
  EGF = pyro.sample('EGF',pyro.distributions.Delta(EGF))
  samples['EGF'] = EGF

  EGFR_N = pyro.sample("EGFR_N", normal_dist)
  EGFR = torch.tensor(check_increase(EGF + EGFR_N))
  EGFR = pyro.sample('EGFR',pyro.distributions.Delta(EGFR))
  samples['EGFR'] = EGFR

  IL_6_N = pyro.sample("IL_6_N", normal_dist)
  IL_6 = torch.tensor(check_increase(ADAM17 + IL_6_N))
  IL_6 = pyro.sample('IL_6',pyro.distributions.Delta(IL_6))
  samples['IL_6'] = IL_6

  IL6_STAT3_N = pyro.sample("IL6_STAT3_N", normal_dist)
  IL6_STAT3 = torch.tensor(check_increase(IL_6 + IL6_STAT3_N))
  IL6_STAT3 = pyro.sample('IL6_STAT3',pyro.distributions.Delta(IL6_STAT3))
  samples['IL6_STAT3'] = IL6_STAT3

# TNF PRR EGFR (ATRI while there is a edge in the graph) code as AND gate
  NF_xB_N = pyro.sample("NF_xB_N", normal_dist)
  if TNF + PRR + EGFR + NF_xB_N > 2.5:    # the threshold should be determined carefully
    NF_xB = torch.tensor(1.0)
  else: 
    NF_xB = torch.tensor(0.0)
  NF_xB = pyro.sample('NF_xB',pyro.distributions.Delta(NF_xB))
  samples['NF_xB'] = NF_xB

# NFXB IL6STAT3 
  IL_6_AMP_N = pyro.sample("IL_6_AMP_N", normal_dist)
  if IL6_STAT3 + NF_xB + IL_6_AMP_N > 1.5:   # the threshold should be determined carefully
    IL_6_AMP = torch.tensor(1.0)
  else: 
    IL_6_AMP = torch.tensor(0.0)
  IL_6_AMP = pyro.sample('IL_6_AMP',pyro.distributions.Delta(IL_6_AMP))
  samples['IL_6_AMP'] = IL_6_AMP

  ARDS_N = pyro.sample("ARDS_N", normal_dist)
  ARDS = torch.tensor(check_increase(IL_6_AMP + ARDS_N))
  ARDS = pyro.sample('ARDS',pyro.distributions.Delta(ARDS))
  samples['ARDS'] = ARDS

  return samples

In [0]:
pyro.set_rng_seed(101)
SCM_model()

{'ACE2': tensor(0.),
 'ADAM17': tensor(1.),
 'ARDS': tensor(0.),
 'ATIR': tensor(1.),
 'AngII': tensor(1.7264),
 'EGF': tensor(1.),
 'EGFR': tensor(1.),
 'IL6_STAT3': tensor(1.),
 'IL_6': tensor(1.),
 'IL_6_AMP': tensor(0.),
 'NF_xB': tensor(0.),
 'PRR': tensor(0.),
 'SARS_2': tensor(0.4908),
 'TNF': tensor(1.),
 'deg': tensor(0.7879)}

In [0]:
from pyro.infer import Importance, EmpiricalMarginal
from torch.distributions.transforms import AffineTransform
# counterfactual function to get the distribution of target variable by sampling
def counterfactual(model,observed_variable, observed_val,exogenous_variable,cf_variable,cf_val,target_variable):
    target = []
    # get the conditional model
    conditions = {}
    for i in range(len(observed_variable)):
        conditions[observed_variable[i]] = torch.tensor(observed_val[i])
    conditioned_model = pyro.condition(model, data=conditions)
    # get the posterior and marginal distribution
    posterior = pyro.infer.Importance(conditioned_model, num_samples=1000).run()
    marginal = EmpiricalMarginal(posterior,exogenous_variable)
    # get the do model
    dos = {}
    for i in range(len(cf_variable)):
        dos[cf_variable[i]] = torch.tensor(cf_val[i])
    do_model = pyro.do(model, data=dos)
    # sample target variable under updated posterior and do model
    for i in range(1000):
        samples = marginal()
        exog = {}
        for i in range(len(exogenous_variable)):
            exog[exogenous_variable[i]] = torch.tensor(samples[i])
        cf_model = pyro.condition(do_model, data= exog)
        trace_handler = pyro.poutine.trace(cf_model)
        trace = trace_handler.get_trace()

        target.append(trace.nodes[target_variable]['value'].numpy().item())

    return np.mean(target)

One thing we notice is that when ACE-2 is inhibited, we see Interleukin 6 (IL-6) activation, which is a pro-inflammatory cytokine that leads to Acute respiratory distress syndrome (ARDS).   Furthermore, there are several drugs that affect intermediates in the pathway from ACE-2 to IL-6.   First question: does the current covid-19 knowledge graph contain this information?  Second:  Can we generate a simulation of ACE-2 inhibition leading to IL-6 activation?   Third, can we identify a protein in the pathway from ACE-2 to IL-6 that is the target of a drug?  Fourth question, conditioned on what we know about normal ACE-2 to IL-6 pathway, what is the effect of ACE-2 inhibition on IL-6 activation when we simulate drug-induced knockout of the protein target?

In [0]:
target = counterfactual(SCM_model, ['SARS_2', 'ACE2','ARDS'], [1.,0.,1.], ['SARS_N', 'PRR_N', 'deg_N', 
                                                     'ACE2_N', 'AngII_N', 'ATIR_N', 
                                                     'ADAM17_N', 'TNF_N', 'EGF_N', 
                                                     'EGFR_N', 'IL_6_N', 'IL6_STAT3_N', 
                                                     'NF_xB_N', 'IL_6_AMP_N','ARDS_N' ],
                ['ACE2'], [1.], 'ARDS' )

print(target)

# SARS_2=1, ACE_2 = 0 => ARDS = 1 

## Can we generate a simulation of ACE-2 inhibition leading to IL-6 activation? 
## this is the intervention problem? basically P(IL_6 = 1| do(ACE_2 = 0))

## Third, can we identify a protein in the pathway from ACE-2 to IL-6 that is the target of a drug
## Ask Jeremy

## 4th conditioned on what we know about normal ACE-2 to IL-6 pathway, what is the effect of ACE-2
## inhibition on IL-6 activation when we simulate drug-induced knockout of the protein target
## 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.204


In [0]:
target = counterfactual(SCM_model, ['SARS_2', 'ACE2','ARDS'], [1.,0.,1.], ['SARS_N', 'PRR_N', 'deg_N', 
                                                     'ACE2_N', 'AngII_N', 'ATIR_N', 
                                                     'ADAM17_N', 'TNF_N', 'EGF_N', 
                                                     'EGFR_N', 'IL_6_N', 'IL6_STAT3_N', 
                                                     'NF_xB_N', 'IL_6_AMP_N','ARDS_N' ],
                ['ACE2'], [1.], 'ARDS' )
print(target)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.044


In [0]:
target = counterfactual(SCM_model, ['SARS_2', 'ACE2','ARDS'], [1.,0.,1.], ['SARS_N', 'PRR_N', 'deg_N', 
                                                     'ACE2_N', 'AngII_N', 'ATIR_N', 
                                                     'ADAM17_N', 'TNF_N', 'EGF_N', 
                                                     'EGFR_N', 'IL_6_N', 'IL6_STAT3_N', 
                                                     'NF_xB_N', 'IL_6_AMP_N','ARDS_N' ],
                ['IL_6'], [0.], 'ARDS' )

print(target)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.0


In [0]:
target = counterfactual(SCM_model, ['SARS_2', 'ACE2','ARDS'], [1.,0.,1.], ['SARS_N', 'PRR_N', 'deg_N', 
                                                     'ACE2_N', 'AngII_N', 'ATIR_N', 
                                                     'ADAM17_N', 'TNF_N', 'EGF_N', 
                                                     'EGFR_N', 'IL_6_N', 'IL6_STAT3_N', 
                                                     'NF_xB_N', 'IL_6_AMP_N','ARDS_N' ],
                ['IL6_STAT3'], [0.], 'ARDS' )

print(target)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.0


In [0]:
# the intervention probability
def intervention(model,do_variable, do_val, target_variable):
    # get the conditions for the do model
    conditions = {}
    for i in range(len(do_variable)):
        conditions[do_variable[i]] = torch.tensor(do_val[i])
    do_model = pyro.do(model, data=conditions)
    posterior = pyro.infer.Importance(do_model, num_samples=1000).run()
    marginal = EmpiricalMarginal(posterior,target_variable)
    target = [marginal().item() for i in range(1000)]
    return np.mean(target)               

In [0]:
intervention(SCM_model,['ACE2'],[0.0],'ARDS')

0.814

In [0]:
intervention(SCM_model,['PRR'],[1.0],'ARDS')

0.87

In [0]:
intervention(SCM_model,['ACE2'],[1.0],'ARDS')

0.138

In [0]:
## what would ARDS be if IL_6_STAT3 is set to low?
intervention(SCM_model, ['IL6_STAT3'], [0.0], 'ARDS')

0.0

In [0]:
### what would ARDS be if PRR is set to low
intervention(SCM_model, ['PRR'], [0.0], 'ARDS')

0.0

In [0]:
intervention(SCM_model, ['PRR'], [0.0], 'ARDS')

In [0]:
# One of the major pathways for NF-κB activation after coronavirus infection
# is the MyD88 pathway through pattern recognition receptors (PRRs), leading to the
# induction of a variety of pro-inflammatory cytokines, including interleukin-6 (IL-6),
# tumor necrosis factor alpha (TNFα) and chemokines

target = counterfactual(SCM_model, ['PRR', 'NF_xB','ARDS'], [1.,1.,1.], ['SARS_N', 'PRR_N', 'deg_N', 
                                                     'ACE2_N', 'AngII_N', 'ATIR_N', 
                                                     'ADAM17_N', 'TNF_N', 'EGF_N', 
                                                     'EGFR_N', 'IL_6_N', 'IL6_STAT3_N', 
                                                     'NF_xB_N', 'IL_6_AMP_N','ARDS_N' ],
                ['PRR'], [0.], 'ARDS' )
print(np.mean(target))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.0


In [0]:
#  But even if you just have the true DGP, you can already ask counterfactual questions. 
#  For example,set the parameters so that when when SARS-CoV-2 is high, ACE-2 is low ,
#   and ARDS is high.  Now, ask the question, what would ARDS be if IL-6-STAT3 or 
#   sIL-6R\alpha was set to low (perhaps by a drug)

But even if you just have the true DGP, you can already ask counterfactual questions.  For example,set the parameters so that when when SARS-CoV-2 is high, ACE-2 is low , and ARDS is high.  Now, ask the question, what would ARDS be if IL-6-STAT3 or sIL-6R\alpha was set to low (perhaps by a drug)